In [1]:
import numpy as np
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# Khởi tạo trình duyệt Chrome
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get("https://vnexpress.net/goc-nhin")  # Điều hướng đến trang Góc Nhìn trên VnExpress
driver.maximize_window()  # Phóng to cửa sổ trình duyệt

def scroll_page(scroll_count=10, pause=1.5, offset=25):
    """
    Cuộn trang để tải thêm nội dung.

    Args:
        scroll_count (int): Số lần cuộn trang.
        pause (float): Thời gian dừng giữa các lần cuộn (đơn vị: giây).
        offset (int): Khoảng cách chừa lại ở cuối trang (tính bằng pixel).
    """
    total_height = driver.execute_script("return document.body.scrollHeight")  # Lấy chiều cao tổng thể của trang
    scroll_step = (total_height - offset) / scroll_count  # Tính khoảng cách cuộn mỗi lần

    for i in range(1, scroll_count + 1):
        driver.execute_script(f"window.scrollTo(0, {i * scroll_step});")  # Cuộn xuống
        sleep(pause)  # Tạm dừng để nội dung tải thêm

def collect_titles_and_links(limit=1000):
    """
    Thu thập tiêu đề, liên kết bài viết, Author, Category, và Comments Count.

    Args:
        limit (int): Số lượng bài viết cần thu thập.

    Returns:
        list: Danh sách các bài viết với thông tin cơ bản.
    """
    collected_data = []  # Danh sách lưu dữ liệu bài viết
    collected_links = set()  # Tập hợp để tránh trùng lặp liên kết
    
    while len(collected_links) < limit:
        articles = driver.find_elements(By.CSS_SELECTOR, "article.item-news")  # Tìm các bài viết trên trang hiện tại
        
        if not articles:  # Nếu không có bài viết nào
            scroll_page(scroll_count=10)  # Cuộn thêm trang
            continue
        
        for article in articles:
            try:
                # Lấy tiêu đề và liên kết bài viết
                title_element = article.find_element(By.CSS_SELECTOR, "h3.title-news a")
                title = title_element.text
                link = title_element.get_attribute("href")
                
                if link not in collected_links:
                    collected_links.add(link)  # Thêm liên kết vào tập hợp để tránh trùng lặp
                    
                    # Lấy thông tin tác giả
                    try:
                        author_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.name-author")
                        author = author_element.text
                        author_link = author_element.get_attribute("href")
                    except NoSuchElementException:
                        author = "Unknown"
                        author_link = None

                    # Lấy thông tin danh mục
                    try:
                        category_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.cat")
                        category = category_element.text
                        category_link = category_element.get_attribute("href")
                    except NoSuchElementException:
                        category = "Unknown"
                        category_link = None

                    # Lấy số lượng bình luận
                    try:
                        comments_count_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.count_cmt span.font_icon")
                        comments_count = comments_count_element.text
                    except NoSuchElementException:
                        comments_count = "0"

                    # Lưu thông tin bài viết vào danh sách
                    collected_data.append({
                        "Title": title,
                        "Link": link,
                        "Author": author,
                        "Author Link": author_link,
                        "Category": category,
                        # "Category Link": category_link,
                        "Comments Count": comments_count
                    })
                    print(f"Collected: {len(collected_links)}/{limit} articles")
                    
                    if len(collected_links) >= limit:  # Dừng lại nếu đủ số lượng bài viết
                        break
            except NoSuchElementException:
                continue
        
        scroll_page(scroll_count=5)  # Cuộn thêm để tải các bài viết mới

    return collected_data

# Hàm thu thập thông tin chi tiết bài viết
def extract_detail_info():
    """
    Thu thập thông tin chi tiết từ một bài viết.

    Returns:
        dict: Thông tin chi tiết của bài viết, bao gồm ngày đăng, nội dung, bình luận.
    """
    try:
        # Lấy thông tin ngày đăng bài
        date_element = driver.find_element(By.CSS_SELECTOR, "span.date")
        date = date_element.text

        # Lấy tiêu đề chi tiết
        title_detail_element = driver.find_element(By.CSS_SELECTOR, "h1.title-detail")
        detailed_title = title_detail_element.text

        # Lấy mô tả bài viết
        description_element = driver.find_element(By.CSS_SELECTOR, "p.description")
        detailed_description = description_element.text

        # Lấy nội dung bài viết
        content_paragraphs = driver.find_elements(By.CSS_SELECTOR, "article.fck_detail p")
        content = "\n".join([paragraph.text for paragraph in content_paragraphs])

        scroll_page(scroll_count=10, pause=1.5, offset=50)  # Cuộn trang để tải toàn bộ nội dung

        # Thu thập bình luận
        comments_data = []
        total_comments_count = 0

        while True:  # Nhấn vào "Xem thêm ý kiến" nếu tồn tại
            try:
                view_more_button = driver.find_element(By.CSS_SELECTOR, "div.view_more_coment a#show_more_coment")
                driver.execute_script("arguments[0].click();", view_more_button)
                sleep(2)
                driver.execute_script("window.scrollBy(0, 500);")
                sleep(1)
            except NoSuchElementException:
                break

        comments = driver.find_elements(By.CSS_SELECTOR, "div.comment_item")
        for comment in comments:
            try:
                # Lấy thông tin nickname và liên kết
                nickname_element = comment.find_element(By.CSS_SELECTOR, "span.txt-name a.nickname")
                nickname = nickname_element.text
                nickname_href = nickname_element.get_attribute("href")

                # Lấy nội dung bình luận
                comment_content_element = comment.find_element(By.CSS_SELECTOR, "p.full_content")
                comment_content = comment_content_element.text.strip()

                # Lấy thời gian bình luận
                time_element = comment.find_element(By.CSS_SELECTOR, "span.time-com")
                comment_time = time_element.text

                # Lấy số lượng like
                try:
                    like_element = comment.find_element(By.CSS_SELECTOR, "div.reactions-total a.number")
                    likes = like_element.text
                except NoSuchElementException:
                    likes = "0"

                # Lấy thông tin trả lời
                reply_nicknames, reply_links, reply_comments = [], [], []
                try:
                    reply_button = comment.find_element(By.CSS_SELECTOR, "a.view_all_reply")
                    driver.execute_script("arguments[0].click();", reply_button)
                    sleep(1)
                    reply_items = comment.find_elements(By.CSS_SELECTOR, "div.sub_comment div.comment_item")
                    for reply in reply_items:
                        try:
                            reply_nickname_element = reply.find_element(By.CSS_SELECTOR, "span.txt-name a.nickname")
                            reply_nicknames.append(reply_nickname_element.text)
                            reply_links.append(reply_nickname_element.get_attribute("href"))

                            reply_content_element = reply.find_element(By.CSS_SELECTOR, "p.full_content")
                            reply_comments.append(reply_content_element.text.strip())
                        except NoSuchElementException:
                            continue
                except NoSuchElementException:
                    pass

                comments_data.append({
                    "Nickname": nickname,
                    "Nickname Link": nickname_href,
                    "Comment": comment_content,
                    "Likes": likes,
                    "Comment Time": comment_time,
                    "Reply Nicknames": reply_nicknames,
                    "Reply Links": reply_links,
                    "Reply Comments": reply_comments
                })
                total_comments_count += 1
            except NoSuchElementException:
                continue

        print(f"Tổng số bình luận và trả lời đã thu thập: {total_comments_count}")

        return {
            "Date": date,
            "Detailed Title": detailed_title,
            "Author's Position": detailed_description,
            "Content": content,
            "Comments_Post": comments_data,
            "Total_Comments": total_comments_count
        }
    except NoSuchElementException:
        return {}


# Thu thập thông tin chi tiết từ danh sách bài viết
def collect_details(articles):
    """
    Thu thập thông tin chi tiết từ danh sách bài viết.

    Args:
        articles (list): Danh sách bài viết với tiêu đề và liên kết.

    Returns:
        list: Danh sách bài viết với thông tin chi tiết.
    """
    detailed_data = []
    
    for idx, article in enumerate(articles):
        print(f"Processing {idx + 1}/{len(articles)}: {article['Link']}")
        driver.get(article["Link"])  # Điều hướng đến bài viết
        sleep(2)
        detail_info = extract_detail_info()
        
        if detail_info:
            detailed_data.append({
                **article,
                **detail_info
            })
    
    return detailed_data

# Bước 1: Thu thập tiêu đề và liên kết
articles = collect_titles_and_links(limit=5)

# Bước 2: Thu thập thông tin chi tiết
detailed_articles = collect_details(articles)

# Lưu dữ liệu vào DataFrame
df = pd.DataFrame(detailed_articles)

# Hiển thị DataFrame
print(df)


Collected: 1/10 articles
Collected: 2/10 articles
Collected: 3/10 articles
Collected: 4/10 articles
Collected: 5/10 articles
Collected: 6/10 articles
Collected: 7/10 articles
Collected: 8/10 articles
Collected: 9/10 articles
Collected: 10/10 articles
Processing 1/10: https://vnexpress.net/mien-hoc-phi-nganh-y-4833819.html
Tổng số bình luận và trả lời đã thu thập: 19
Processing 2/10: https://vnexpress.net/den-viet-nam-chua-benh-4832553.html
Tổng số bình luận và trả lời đã thu thập: 52
Processing 3/10: https://vnexpress.net/an-cap-thoi-cong-nghe-4833205.html
Tổng số bình luận và trả lời đã thu thập: 14
Processing 4/10: https://vnexpress.net/nao-ca-vang-vui-ve-4832631.html
Tổng số bình luận và trả lời đã thu thập: 68
Processing 5/10: https://vnexpress.net/tinh-gian-giay-to-don-xin-4832462.html
Tổng số bình luận và trả lời đã thu thập: 30
Processing 6/10: https://vnexpress.net/ve-metro-mien-phi-4832075.html
Tổng số bình luận và trả lời đã thu thập: 40
Processing 7/10: https://vnexpress.net

In [3]:
# Lưu vào file Excel
df.to_excel("D:/du lieu o cu/HUTECH Courses/Social Networking Course/SocialNetworkingProject/Project của Đạt/vnexpress_articles_3.xlsx")
